<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# SQL with Python

by [The Carpentries](https://training.galaxyproject.org/hall-of-fame/carpentries/), [Helena Rasche](https://training.galaxyproject.org/hall-of-fame/hexylena/), [Avans Hogeschool](https://training.galaxyproject.org/hall-of-fame/avans-atgm/)

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- How can I access databases from programs written in Python?

**Objectives**

- Write short programs that execute SQL queries.
- Trace the execution of a program that contains an SQL query.
- Explain why most database applications are written in a general-purpose language rather than in SQL.

**Time Estimation: 45M**
</div>


<blockquote class="comment" style="border: 2px solid #ffecc1; margin: 1em 0.2em">
<div class="box-title" aria-label="comment box: Comment" style="font-size: 150%">💬 Comment</div>
<p>This tutorial is <strong>significantly</strong> based on <a href="https://carpentries.org">the Carpentries</a> <a href="https://github.com/swcarpentry/sql-novice-survey/">Databases and SQL</a> lesson, which is licensed CC-BY 4.0.</p>
<p>Abigail Cabunoc and Sheldon McKay (eds): “Software Carpentry: Using Databases and SQL.”  Version 2017.08, August 2017,
<a href="https://github.com/swcarpentry/sql-novice-survey">github.com/swcarpentry/sql-novice-survey</a>, <a href="https://doi.org/10.5281/zenodo.838776">https://doi.org/10.5281/zenodo.838776</a></p>
<p>Adaptations have been made to make this work better in a GTN/Galaxy environment.</p>
</blockquote>
<blockquote class="agenda" style="border: 2px solid #86D486;display: none; margin: 1em 0.2em">
<div class="box-title" aria-label="agenda box: Agenda" style="font-size: 150%"> Agenda</div>
<p>In this tutorial, we will cover:</p>
</blockquote>
<p>For this tutorial we need to download a database that we will use for the queries.</p>


In [ ]:
!wget -c http://swcarpentry.github.io/sql-novice-survey/files/survey.db

<h1 id="programming-with-databases---python">Programming with Databases - Python</h1>
<p>Let’s have a look at how to access a database from
a general-purpose programming language like Python.
Other languages use almost exactly the same model:
library and function names may differ,
but the concepts are the same.</p>
<p>Here’s a short Python program that selects latitudes and longitudes
from an SQLite database stored in a file called <code style="color: inherit">survey.db</code>:</p>


In [ ]:
import sqlite3

connection = sqlite3.connect("survey.db")
cursor = connection.cursor()
cursor.execute("SELECT Site.lat, Site.long FROM Site;")
results = cursor.fetchall()
for r in results:
    print(r)
cursor.close()
connection.close()

<p>The program starts by importing the <code style="color: inherit">sqlite3</code> library.
If we were connecting to MySQL, DB2, or some other database,
we would import a different library,
but all of them provide the same functions,
so that the rest of our program does not have to change
(at least, not much)
if we switch from one database to another.</p>
<p>Line 2 establishes a connection to the database.
Since we’re using SQLite,
all we need to specify is the name of the database file.
Other systems may require us to provide a username and password as well.
Line 3 then uses this connection to create a cursor.
Just like the cursor in an editor,
its role is to keep track of where we are in the database.</p>
<p>On line 4, we use that cursor to ask the database to execute a query for us.
The query is written in SQL,
and passed to <code style="color: inherit">cursor.execute</code> as a string.
It’s our job to make sure that SQL is properly formatted;
if it isn’t,
or if something goes wrong when it is being executed,
the database will report an error.</p>
<p>The database returns the results of the query to us
in response to the <code style="color: inherit">cursor.fetchall</code> call on line 5.
This result is a list with one entry for each record in the result set;
if we loop over that list (line 6) and print those list entries (line 7),
we can see that each one is a tuple
with one element for each field we asked for.</p>
<p>Finally, lines 8 and 9 close our cursor and our connection,
since the database can only keep a limited number of these open at one time.
Since establishing a connection takes time,
though,
we shouldn’t open a connection,
do one operation,
then close the connection,
only to reopen it a few microseconds later to do another operation.
Instead,
it’s normal to create one connection that stays open for the lifetime of the program.</p>
<p>Queries in real applications will often depend on values provided by users.
For example,
this function takes a user’s ID as a parameter and returns their name:</p>


In [ ]:
import sqlite3

def get_name(database_file, person_id):
    query = "SELECT personal || ' ' || family FROM Person WHERE id='" + person_id + "';"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    return results[0][0]

print("Full name for dyer:", get_name('survey.db', 'dyer'))

<p>We use string concatenation on the first line of this function
to construct a query containing the user ID we have been given.
This seems simple enough,
but what happens if someone gives us this string as input?</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">dyer'; DROP TABLE Survey; SELECT '
</code></pre></div></div>
<p>It looks like there’s garbage after the user’s ID,
but it is very carefully chosen garbage.
If we insert this string into our query,
the result is:</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">SELECT personal || ' ' || family FROM Person WHERE id='dyer'; DROP TABLE Survey; SELECT '';
</code></pre></div></div>
<p>If we execute this,
it will erase one of the tables in our database.</p>
<p>This is called an SQL injection attack,
and it has been used to attack thousands of programs over the years.
In particular,
many web sites that take data from users insert values directly into queries
without checking them carefully first.
A very <a href="https://xkcd.com/327/">relevant XKCD</a> that explains the
dangers of using raw input in queries a little more succinctly:</p>
<p><img src="...
QmCC
"" alt="A 4 panel comic, in the first panel a person is shown answering the phone, hearing that their son's school has some computer trouble. In panel 2 they apologises asking if their child broke something. In panel 3, the unseen person on the other end of the phone call asks if they really named their son Robert'); Drop table students;--? They respond saying 'oh yes. little bobby tables we call him.' In the 4th panel the caller says 'well we have lost this years student records, I hope you're happy.' They respond 'And I hope you've learned to sanitize your database inputs'." loading="lazy" /></p>
<p>Since a villain might try to smuggle commands into our queries in many different ways,
the safest way to deal with this threat is
to replace characters like quotes with their escaped equivalents,
so that we can safely put whatever the user gives us inside a string.
We can do this by using a prepared statement
instead of formatting our statements as strings.
Here’s what our example program looks like if we do this:</p>


In [ ]:
import sqlite3

def get_name(database_file, person_id):
    query = "SELECT personal || ' ' || family FROM Person WHERE id=?;"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query, [person_id])
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    return results[0][0]

print("Full name for dyer:", get_name('survey.db', 'dyer'))

<p>The key changes are in the query string and the <code style="color: inherit">execute</code> call.
Instead of formatting the query ourselves,
we put question marks in the query template where we want to insert values.
When we call <code style="color: inherit">execute</code>,
we provide a list
that contains as many values as there are question marks in the query.
The library matches values to question marks in order,
and translates any special characters in the values
into their escaped equivalents
so that they are safe to use.</p>
<p>We can also use <code style="color: inherit">sqlite3</code>’s cursor to make changes to our database,
such as inserting a new name.
For instance, we can define a new function called <code style="color: inherit">add_name</code> like so:</p>


In [ ]:
import sqlite3

def add_name(database_file, new_person):
    query = "INSERT INTO Person (id, personal, family) VALUES (?, ?, ?);"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query, list(new_person))
    cursor.close()
    connection.close()


def get_name(database_file, person_id):
    query = "SELECT personal || ' ' || family FROM Person WHERE id=?;"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query, [person_id])
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    return results[0][0]

# Insert a new name
add_name('survey.db', ('barrett', 'Mary', 'Barrett'))
# Check it exists
print("Full name for barrett:", get_name('survey.db', 'barrett'))

<p>Note that in versions of sqlite3 &gt;= 2.5, the <code style="color: inherit">get_name</code> function described
above will fail with an <code style="color: inherit">IndexError: list index out of range</code>,
even though we added Mary’s
entry into the table using <code style="color: inherit">add_name</code>.
This is because we must perform a <code style="color: inherit">connection.commit()</code> before closing
the connection, in order to save our changes to the database.</p>


In [ ]:
import sqlite3

def add_name(database_file, new_person):
    query = "INSERT INTO Person (id, personal, family) VALUES (?, ?, ?);"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query, list(new_person))
    cursor.close()
    connection.commit()
    connection.close()


def get_name(database_file, person_id):
    query = "SELECT personal || ' ' || family FROM Person WHERE id=?;"

    connection = sqlite3.connect(database_file)
    cursor = connection.cursor()
    cursor.execute(query, [person_id])
    results = cursor.fetchall()
    cursor.close()
    connection.close()

    return results[0][0]

# Insert a new name
add_name('survey.db', ('barrett', 'Mary', 'Barrett'))
# Check it exists
print("Full name for barrett:", get_name('survey.db', 'barrett'))

<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question: Filling a Table vs. Printing Values" style="font-size: 150%">❓ Question: Filling a Table vs. Printing Values</div>
<p>Write a Python program that creates a new database in a file called
<code style="color: inherit">original.db</code> containing a single table called <code style="color: inherit">Pressure</code>, with a
single field called <code style="color: inherit">reading</code>, and inserts 100,000 random numbers
between 10.0 and 25.0.  How long does it take this program to run?
How long does it take to run a program that simply writes those
random numbers to a file?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<div class="language-plaintext python highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">import sqlite3
# import random number generator
from numpy.random import uniform

random_numbers = uniform(low=10.0, high=25.0, size=100000)

connection = sqlite3.connect("original.db")
cursor = connection.cursor()
cursor.execute("CREATE TABLE Pressure (reading float not null)")
query = "INSERT INTO Pressure (reading) VALUES (?);"

for number in random_numbers:
    cursor.execute(query, [number])

cursor.close()
connection.commit() # save changes to file for next exercise
connection.close()
</code></pre></div>    </div>
<p>For comparison, the following program writes the random numbers
into the file <code style="color: inherit">random_numbers.txt</code>:</p>
<div class="language-plaintext python highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">from numpy.random import uniform

random_numbers = uniform(low=10.0, high=25.0, size=100000)
with open('random_numbers.txt', 'w') as outfile:
    for number in random_numbers:
        # need to add linebreak \n
        outfile.write("{}\n".format(number))
</code></pre></div>    </div>
</details>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question: Filtering in SQL vs. Filtering in Python" style="font-size: 150%">❓ Question: Filtering in SQL vs. Filtering in Python</div>
<p>Write a Python program that creates a new database called
<code style="color: inherit">backup.db</code> with the same structure as <code style="color: inherit">original.db</code> and copies all
the values greater than 20.0 from <code style="color: inherit">original.db</code> to <code style="color: inherit">backup.db</code>.
Which is faster: filtering values in the query, or reading
everything into memory and filtering in Python?</p>
<br/><details style="border: 2px solid #B8C3EA; margin: 1em 0.2em; padding: 0.5em;"><summary>👁 View solution</summary>
<div class="box-title" aria-label="solution box: Solution" style="font-size: 150%">👁 Solution</div>
<p>The first example reads all the data into memory and filters the
numbers using the if statement in Python.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">import sqlite3

connection_original = sqlite3.connect("original.db")
cursor_original = connection_original.cursor()
cursor_original.execute("SELECT * FROM Pressure;")
results = cursor_original.fetchall()
cursor_original.close()
connection_original.close()

connection_backup = sqlite3.connect("backup.db")
cursor_backup = connection_backup.cursor()
cursor_backup.execute("CREATE TABLE Pressure (reading float not null)")
query = "INSERT INTO Pressure (reading) VALUES (?);"

for entry in results:
    # number is saved in first column of the table
    if entry[0] &gt; 20.0:
        cursor_backup.execute(query, entry)

cursor_backup.close()
connection_backup.commit()
connection_backup.close()
</code></pre></div>    </div>
<p>In contrast the following example uses the conditional <code style="color: inherit"></code>SELECT<code style="color: inherit"></code> statement
to filter the numbers in SQL.
The only lines that changed are in line 5, where the values are fetched
from <code style="color: inherit">original.db</code> and the for loop starting in line 15 used to insert
the numbers into <code style="color: inherit">backup.db</code>.
Note how this version does not require the use of Python’s if statement.</p>
<div class="language-plaintext highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">import sqlite3

connection_original = sqlite3.connect("original.db")
cursor_original = connection_original.cursor()
cursor_original.execute("SELECT * FROM Pressure WHERE reading &gt; 20.0;")
results = cursor_original.fetchall()
cursor_original.close()
connection_original.close()

connection_backup = sqlite3.connect("backup.db")
cursor_backup = connection_backup.cursor()
cursor_backup.execute("CREATE TABLE Pressure (reading float not null)")
query = "INSERT INTO Pressure (reading) VALUES (?);"

for entry in results:
    cursor_backup.execute(query, entry)

cursor_backup.close()
connection_backup.commit()
connection_backup.close()
</code></pre></div>    </div>
</blockquote>
</blockquote>
<blockquote class="question" style="border: 2px solid #8A9AD0; margin: 1em 0.2em">
<div class="box-title" aria-label="question box: Question: Generating Insert Statements" style="font-size: 150%">❓ Question: Generating Insert Statements</div>
<p>One of our colleagues has sent us a
CSV
file containing
temperature readings by Robert Olmstead, which is formatted like
this:</p>
<div class="language-plaintext output highlighter-rouge"><div><pre style="color: inherit; background: transparent"><code style="color: inherit">Taken,Temp
619,-21.5
622,-15.5
</code></pre></div>  </div>
<p>Write a small Python program that reads this file in and prints out
the SQL <code style="color: inherit">INSERT</code> statements needed to add these records to the
survey database.  Note: you will need to add an entry for Olmstead
to the <code style="color: inherit">Person</code> table.  If you are testing your program repeatedly,
you may want to investigate SQL’s <code style="color: inherit">INSERT or REPLACE</code> command.</p>
</blockquote>


# Key Points

- General-purpose languages have libraries for accessing databases.
- To connect to a database, a program must use a library specific to that database manager.
- These libraries use a connection-and-cursor model.
- Programs can read query results in batches or all at once.
- Queries should be written using parameter substitution, not string formatting.

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/sql-python/tutorial.html#feedback) and check there for further resources!
